<a href="https://colab.research.google.com/github/vishwatheerthala/AI/blob/main/A5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd

In [ ]:
data = pd.read_excel('assign5.xlsx')

In [ ]:
data

,Date,Time,Load (kW)
0,01.09.2018,00:00:00,5551.82208
1,NaN,01:00:00,4983.17184
2,NaN,02:00:00,4888.39680
3,NaN,03:00:00,5072.95872
4,NaN,04:00:00,5196.25980
...,...,...,...
2179,NaN,1900-03-28 19:00:00,4626.51840
2180,NaN,1900-03-28 20:00:00,4567.90752
2181,NaN,1900-03-28 21:00:00,4723.63164
2182,NaN,1900-03-28 22:00:00,5150.27520


In [ ]:
lt_3=[]
for i in range(0,data.shape[0]-3):
    lt_3.append(data['Load (kW)'][i])

In [ ]:
lt_2=[]
for i in range(1,data.shape[0]-2):
    lt_2.append(data['Load (kW)'][i])

In [ ]:
lt_1=[]
for i in range(2,data.shape[0]-1):
    lt_1.append(data['Load (kW)'][i])

In [ ]:
lt_0=[]
for i in range(3,data.shape[0]-0):
    lt_0.append(data['Load (kW)'][i])

In [ ]:
lt_3Max=np.max(lt_3)
lt_2Max=np.max(lt_2)
lt_1Max=np.max(lt_1)
lt_0Max=np.max(lt_0)
lt_3Min=np.min(lt_3)
lt_2Min=np.min(lt_2)
lt_1Min=np.min(lt_1)
lt_0Min=np.min(lt_0)
for i in range(len(lt_3)):
    lt_0[i]=(lt_0[i]-lt_0Min)/(lt_0Max-lt_0Min)
    lt_1[i]=(lt_1[i]-lt_1Min)/(lt_1Max-lt_1Min)
    lt_2[i]=(lt_2[i]-lt_2Min)/(lt_2Max-lt_2Min)
    lt_3[i]=(lt_3[i]-lt_3Min)/(lt_3Max-lt_3Min)

In [ ]:
load=pd.DataFrame({'LT-3':lt_3,'LT-2':lt_2,'LT-1':lt_1, 'LT-0':lt_0})

In [ ]:
from sklearn.model_selection import train_test_split
X, y = load.drop('LT-0',axis=1),load['LT-0']
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 10)

In [ ]:
def gradient(m1,m2,m3,c,x1,x2,x3,y):
    m1_deriv=-1*(y-m1*x1-m2*x2-m3*x3-c)*x1
    m2_deriv=-1*(y-m1*x1-m2*x2-m3*x3-c)*x2
    m3_deriv=-1*(y-m1*x1-m2*x2-m3*x3-c)*x3
    c_deriv=-1*(y-m1*x1-m2*x2-m3*x3-c)
    return m1_deriv,m2_deriv,m3_deriv,c_deriv
import random
m1=random.uniform(-10,10)
m2=random.uniform(-10,10)
m3=random.uniform(-10,10)
c=random.uniform(-10,10)
eta=0.1
max_iter=500

for iter in range(max_iter):
    for i in range(x_train.shape[0]):
        del_m1,del_m2,del_m3,del_c=gradient(m1,m2,m3,c,x_train.iat[i,0],x_train.iat[i,1],x_train.iat[i,2],y_train.iloc[i])
        m1=m1-eta*del_m1
        m2=m2-eta*del_m2
        m3=m3-eta*del_m3
        c=c-eta*del_c
print('model parameter m and c',m1,m2,m3,c)

model parameter m and c -0.20929847981904956 0.030441491869554953 0.8551628652827352 0.1467206828032618


In [ ]:
y_pred_train = []
for i in range(x_train.shape[0]):
    x1=x_train.iat[i,0]
    x2=x_train.iat[i,1]
    x3=x_train.iat[i,2]
    y=y_train.iat[i]
    y_pred = (m1 * x1) + c + (m2 * x2) + (m3 * x3)
    y_pred_train.append(y_pred)
y_pred_train = np.array(y_pred_train)

In [ ]:
y_pred_test = []
for i in range(x_test.shape[0]):
    x1=x_test.iat[i,0]
    x2=x_test.iat[i,1]
    x3=x_test.iat[i,2]
    y=y_test.iat[i]
    y_pred = (m1 * x1) + c + (m2 * x2) + (m3 * x3)
    y_pred_test.append(y_pred)
y_pred_test = np.array(y_pred_test)

In [ ]:
y_train_denorm=[]
y_train_pred_denorm=[]
for i in range(y_train.shape[0]):
    y_train_denorm.append(y_train.iloc[i]*(lt_0Max-lt_0Min)+lt_0Min)
    y_train_pred_denorm.append(y_pred_train[i]*(lt_0Max-lt_0Min)+lt_0Min)
y_pred_train=y_train_pred_denorm
y_train=y_train_denorm

In [ ]:
y_test_denorm=[]
y_test_pred_denorm=[]
for i in range(y_test.shape[0]):
    y_test_denorm.append(y_test.iloc[i]*(lt_0Max-lt_0Min)+lt_0Min)
    y_test_pred_denorm.append(y_pred_test[i]*(lt_0Max-lt_0Min)+lt_0Min)
y_pred_test=y_test_pred_denorm
y_test=y_test_denorm

In [ ]:
import math
from sklearn.metrics import mean_squared_error 
from sklearn.metrics import mean_absolute_error
rmse = math.sqrt(mean_squared_error(y_train, y_pred_train)) 
print('Root mean square error', rmse) 
mse = (mean_squared_error(y_train, y_pred_train)) 
print('Mean square error', mse) 
mae=mean_absolute_error(y_train, y_pred_train)
print('Mean absolute error', mae)

Root mean square error 647.1959879937988
Mean square error 418862.6468752693
Mean absolute error 443.2490079981892


In [ ]:
mse = math.sqrt(mean_squared_error(y_test, y_pred_test)) 
print('Root mean square error', mse) 
mse = (mean_squared_error(y_test, y_pred_test)) 
print('Mean square error', mse) 
mae=mean_absolute_error(y_test, y_pred_test)
print('Mean absolute error', mae)

Root mean square error 620.8444142830592
Mean square error 385447.7867464749
Mean absolute error 438.08205424113845
